In [1]:
%%capture
import os
os.chdir('/tmp')
!wget https://github.com/MontrealCorpusTools/Montreal-Forced-Aligner/releases/download/v1.0.1/montreal-forced-aligner_linux.tar.gz
!tar zxvf montreal-forced-aligner_linux.tar.gz
!ln -s /tmp/montreal-forced-aligner/lib/libpython3.6m.so.1.0 /tmp/montreal-forced-aligner/lib/libpython3.6m.so


In [2]:
os.chdir('/kaggle/working')
os.environ['LD_LIBRARY_PATH'] = f'{os.environ["LD_LIBRARY_PATH"]}:/tmp/montreal-forced-aligner/lib/'
os.environ['PATH'] = f'{os.environ["PATH"]}:/tmp/montreal-forced-aligner/bin/'

In [3]:
%%capture
!apt-get -y install libgfortran3

In [4]:
!mkdir example

The example below is from section 488 (p. 239) of [_Gaeilge Chorca Dhuibhne_](https://shop.dias.ie/product/gaeilge-chorca-dhuibhne/) by Diarmuid Ó Sé.

The provided transcription is:
əs kiːn′ l′əm nə ˈheːn′ɪ v′eh ə bɪn′t′ vuːn er′

In [5]:
%%writefile example/test1.lab
is cuimhin liom na haoinne a bheith ag baint mhóna air

Writing example/test1.lab


Ibid, Section 488, p. 238

xuːərˈdiːs ˈgax ɑːt′

In [6]:
%%writefile example/test2.lab
chuardaíos gach áit

Writing example/test2.lab


MFA insists on having `.wav` files, which it reads, even though it makes no use of them for G2P

In [7]:
%%capture
!apt-get -y install sox

In [8]:
!sox -n -r 16000 -b 16 -c 1 -L example/test1.wav trim 0.0 6.000
!sox -n -r 16000 -b 16 -c 1 -L example/test2.wav trim 0.0 6.000

In [9]:
!mfa_generate_dictionary -t tmp ../input/train-irish-mfa-model-fuaimeanna/g2p-munster.zip example/ output

Setting up corpus information...


In [10]:
!cat output

ag	a ɡ
gach	ɡ ə x
a	ə
is	ɪ ʃ
cuimhin	k ɪ vˠ nʲ
na	n̪ˠ ə
bheith	vʲ ɛ
haoinne	ɪ nʲ ɛ
mhóna	vˠ oː n̪ˠ ə
liom	lʲ ʌ mˠ
áit	ɑː tʲ
baint	bˠ ɪ nʲ tʲ
air	a ɾʲ
chuardaíos	x uə ɾˠ d̪ˠ iː ʌ sˠ


| Word | Pronunciation | Alt. Transcript | Generated | Correct? | Rule/Reason |
| -- | -- | -- | -- | -- | -- |
| is | əs | əsˠ | ɪ ʃ | ❌ | Exception: `ios` |
| cuimhin | kiːn′ | kiːnʲ | k ɪ vˠ nʲ | ❌ | Missing grapheme: `uimhi` |
| liom | l′əm | lʲəmˠ (~ lʲʌmˠ) | lʲ ʌ mˠ | ✔️ | (See, e.g., section 291: l′um) |
| na | nə | n̪ˠə | n̪ˠ ə | ✔️ | |
| haoinne | ˈheːn′ɪ | heːnʲɪ | ɪ nʲ ɛ | ❌ | |
| a | ə | | ə | ✔️ | |
| bheith | v′eh | vʲɛh | vʲ ɛ | ✔️ | Section 9: h  → ∅ / `_` # C |
| ag | ə | ə (~ ɪɟ) | a ɡ | ❌ | ɪɟ, section 60|
| baint | bɪn′t′ | bˠɪnʲtʲ | bˠ ɪ nʲ tʲ | ✔️ | |
| mhóna | vuːn | vˠuːn̪ˠ(ə) | vˠ oː n̪ˠ ə | ✔️ | ó → oː ~ uː / `_` [+nasal], ə  → ∅ / `_` # |
| air | er′ | eɾʲ | a ɾʲ | ❌ | Exception: `eir` |
| chuardaíos | xuːərˈdiːs | xuəɾˠd̪ˠiːsˠ | x uə ɾˠ d̪ˠ iː ʌ sˠ | ❌ | Missing grapheme `uío`
| gach | ˈgax | ɡax (~ ɡəx) | ɡ ə x | ✔️ | See section 810 |
| áit | ɑːt′ | ɑːtʲ | ɑː tʲ | ✔️ | |

